In [13]:
# 定义保存数据的类
import numpy as np
from sklearn.preprocessing import StandardScaler

#定义类
class IPAccessInfo(object):
    def __init__(self, X, y, ip_dict, ip_reserve_list, data_scaler):
        self.X = X
        self.y = y
        self.ip_dict = ip_dict
        self.ip_reserve_list = ip_reserve_list
        self.data_scaler = data_scaler

In [14]:
import pickle
fp = open('iPAccessInfo.pkl','rb',True)
iPAccessInfo = pickle.load(fp)

In [15]:
print(iPAccessInfo.X.shape)
print(iPAccessInfo.y.shape)
print(iPAccessInfo.data_scaler)

(42194, 6)
(42194, 6)
StandardScaler(copy=True, with_mean=True, with_std=True)


In [16]:
iPAccessInfo.X

array([[-0.17549354, -0.06472002, -0.25740818, -0.05018498,  0.        ,
         0.        ],
       [ 0.19567413, -0.06472002, -0.25740818, -0.05018498,  0.        ,
         0.        ],
       [-0.09301183, -0.06472002, -0.25740818, -0.05018498,  0.        ,
         0.        ],
       ..., 
       [-0.07239141, -0.06472002, -0.25740818, -0.05018498,  0.        ,
         0.        ],
       [-0.16518332, -0.06472002, -0.25740818, -0.05018498,  0.        ,
         0.        ],
       [-0.18580375, -0.06472002, -0.25740818, -0.05018498,  0.        ,
         0.        ]])

In [21]:
# mount = 5000
# train_X = iPAccessInfo.X[:mount]
# train_y = iPAccessInfo.y[:mount]

# test_X = iPAccessInfo.X[mount:]
# test_y = iPAccessInfo.y[mount:]

from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(iPAccessInfo.X, iPAccessInfo.y, test_size=0.67, random_state=888)

print (train_X.shape, test_X.shape)

(13924, 6) (28270, 6)


In [22]:
# 定义神经网络
# 6个输入，6个输出

from keras.layers import Input, Dense
from keras.models import Model

# This returns a tensor
inputs = Input(shape=(6,))

# a layer instance is callable on a tensor, and returns a tensor
x = Dense(32, activation='relu')(inputs) #第一个隐藏层，使用relu作为激活函数，输出矩阵为1*32?
x = Dense(32, activation='relu')(x)#第二个隐藏层，输入为上一个隐层的输出，激活函数为relu，输出矩阵为
predictions = Dense(6, activation='softmax')(x)#输出层，6个输出，使用softmax作为最终的激活函数

# This creates a model that includes
# the Input layer and two Dense layers
model = Model(inputs=inputs, outputs=predictions)
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

hist = model.fit(train_X, train_y, verbose=1,
         batch_size=32, epochs=50, validation_data=(test_X, test_y))  # starts training

# model.fit(train_X, train_y, verbose=1,
#           batch_size=32, epochs=50)  # starts training

Train on 13924 samples, validate on 28270 samples
Epoch 1/50
13924/13924 [==============================] - 0s - loss: 0.0908 - acc: 0.9726 - val_loss: 0.0208 - val_acc: 0.9942
Epoch 2/50
13924/13924 [==============================] - 0s - loss: 0.0199 - acc: 0.9961 - val_loss: 0.0204 - val_acc: 0.9984
Epoch 3/50
13924/13924 [==============================] - 0s - loss: 0.0188 - acc: 0.9966 - val_loss: 0.0190 - val_acc: 0.99470.99
Epoch 4/50
13924/13924 [==============================] - 0s - loss: 0.0180 - acc: 0.9963 - val_loss: 0.0194 - val_acc: 0.9979
Epoch 5/50
13924/13924 [==============================] - 0s - loss: 0.0177 - acc: 0.9966 - val_loss: 0.0183 - val_acc: 0.9947
Epoch 6/50
13924/13924 [==============================] - 0s - loss: 0.0170 - acc: 0.9963 - val_loss: 0.0191 - val_acc: 0.9944
Epoch 7/50
13924/13924 [==============================] - 0s - loss: 0.0165 - acc: 0.9967 - val_loss: 0.0173 - val_acc: 0.9987
Epoch 8/50
13924/13924 [==============================] -

In [23]:
#每一个类别，进行识别率和F1-score验证
pred_y = model.predict(test_X, batch_size=32, verbose=0)

pred_idx = np.argmax(pred_y,axis=1)
test_idx = np.argmax(test_y,axis=1)

In [24]:
from sklearn.metrics import classification_report

print (classification_report(test_idx, pred_idx))
# print (test_generator.class_indices)

             precision    recall  f1-score   support

          0       1.00      1.00      1.00     26482
          1       1.00      0.04      0.07       111
          2       0.93      1.00      0.96      1602
          3       1.00      1.00      1.00        75

avg / total       1.00      1.00      0.99     28270

